# CS486 - Artificial Intelligence
## Lesson 8 - Simulated Annealing

Annealing is the heating of a metal and then slowly cooling it at a controlled rate, usually to make it more malleable. Simulated annealing is an algorihm that tries random solutions and becomes less and less random as it "cools". 

Walk through the [simulated annealing examples on the AIMA webise](http://aimacode.github.io/aima-javascript/4-Beyond-Classical-Search/) to get a better idea of how it works. 

## Traveling Salesman Problem

Imagine there is a set of cities that you need to visit. How can you plan you route to minimize the distance travelled between them? This is the Traveling Salesman Problem and it is NP-hard. Let's apply simulated annealing to TSP. 

In [ ]:
from helpers import *
from aima.search import *
from aima.notebook import psource

Let's define TSP an AIMA `Problem`:

In [ ]:
import networkx as nx
import matplotlib.pyplot as plt

class TSP(Problem):
    # we need distances for computing the cost of a state
    # and locations for displaying the map
    def __init__(self, initial, distances, locations):
        self.distances = distances
        self.locations = locations
        Problem.__init__(self,initial)
    
    # randomly flip sections of the current state
    def random_path(self,state):
        state2 = state[:]
        l = random.randint(0, len(state2) - 1)
        r = random.randint(0, len(state2) - 1)

        if l > r:
            l, r = r,l
        
        state2[l : r + 1] = reversed(state2[l : r + 1])  
        return state2
    
    # the only action to take is to try another random path
    def actions(self, state):
        return [self.random_path]
    
    # the only result is the next random path
    def result(self, state, action):
        return action(state)

    # states cost the sum of the distances of every path 
    def value(self, state):
        cost = 0
        for i in range(len(state) - 1):
            cost -= self.distances[state[i]][state[i + 1]]
        cost -= self.distances[state[0]][state[-1]]
        return cost

    # print a map so we can see the map
    def display(self,state):
        f = plt.figure(figsize=(15,6))
        pairs = {state[j-1]: [state[j]] for j in range(len(state))}

        G = nx.Graph(pairs)
        nx.draw(G, pos=self.locations,node_color='none')
        
        # print cost of state in upper right
        plt.gcf().text(1, 1, str(int(self.value(state))), fontsize=24)
        
        # add a white bounding box behind the node labels
        label = nx.draw_networkx_labels(G, pos=self.locations, font_size=14, label="Foo")
        [l.set_bbox(dict(facecolor='white', edgecolor='none')) for l in label.values()]

We'll use the cities from our Romania problem for our TSP. Assume a solution is a list of cities and you visit cities in the order they appear in the list. 

For our initial state, we'll visit the cities in alphabetical order: 

> Arad $\rightarrow$ Bucharest $\rightarrow$ ... $\rightarrow$ Vaslui $\rightarrow$ Zerind  $\rightarrow$ Arad

In [ ]:
tsp = TSP(romania.cities, romania.distances, romania.locations)
tsp.display(tsp.initial)

Now let's approximate a solution using simulated annealing. First, we need a *schedule* that controls how long and fast the simulation cools. The schedule takes three parameters: An initial temperature, a `lambda` that determines the cool rate, the maximum number of iterations. 

In [ ]:
psource(exp_schedule)
schedule = exp_schedule(k=100, lam=0.03, limit=1500)

We have everything to run the simulation and see how it did:

In [ ]:
tsp.display(simulated_annealing(tsp,schedule))

To get an idea of how the simulation arrived at that solution, watch the visualization below:

In [ ]:
import time
from IPython import display

# returns all intermediate states
results = simulated_annealing_full(tsp,schedule)

for i in range(0,len(results)):
    if results[i] != results[i-1]:
        tsp.display(results[i])
        display.clear_output(wait=True)
        plt.show()
        time.sleep(0.25)